In [33]:
import pandas as pd

# Load specific sheet into a DataFrame
df = pd.read_excel('TST00.xlsx', sheet_name='CALENDAR')

# Select specific column within DataFrame
clndr_data = df['clndr_data']

# Convert the column data to string
clndr_data = clndr_data.astype(str)

# Remove the character with unicode value = 127
clndr_data = clndr_data.replace(chr(127), '')

# Convert Series to one long string
clndr_data_string = ''.join(clndr_data)

# Remove '\x7f' from the string
clndr_data_string = clndr_data_string.replace('\x7f', '')

# Remove spaces from the string
clndr_data_string = clndr_data_string.replace(' ', '')

In [34]:
clndr_data_string

'(0||CalendarData()((0||DaysOfWeek()((0||1()((0||0(s|07:00|f|17:00)())))(0||2()((0||0(s|07:00|f|17:00)())))(0||3()((0||0(s|07:00|f|17:00)())))(0||4()((0||0(s|07:00|f|17:00)())))(0||5()((0||0(s|07:00|f|17:00)())))(0||6()())(0||7()((0||0(s|07:00|f|17:00)())))))(0||VIEW(ShowTotal|Y)())(0||Exceptions()((0||0(d|44197)((0||0(s|08:00|f|16:00)())))(0||1(d|44979)())(0||2(d|45038)())(0||3(d|45039)())(0||4(d|45040)())(0||5(d|45041)())(0||6(d|45104)())(0||7(d|45105)())(0||8(d|45106)())(0||9(d|45108)())(0||10(d|45192)())(0||11(d|45344)())(0||12(d|45391)())(0||13(d|45392)())(0||14(d|45393)())(0||15(d|45395)())(0||16(d|45458)())(0||17(d|45459)())(0||18(d|45460)())(0||19(d|45461)())(0||20(d|45558)())(0||21(d|45679)())(0||22(d|45710)())(0||23(d|45746)())(0||24(d|45747)())(0||25(d|45748)())(0||26(d|45749)())(0||27(d|45813)())(0||28(d|45815)())(0||29(d|45816)())(0||30(d|45817)())(0||31(d|45923)())(0||32(d|46075)())(0||33(d|46100)())(0||34(d|46102)())(0||35(d|46103)())(0||36(d|46104)())(0||37(d|46168)())(

In [35]:
# Find start and end indices of required segment
start = clndr_data_string.find("CalendarData()") + len("CalendarData()")
end = clndr_data_string.rfind("(0||VIEW(ShowTotal|Y)())")

# Extract the segment by slicing the string
segment = clndr_data_string[start:end]
segment

'((0||DaysOfWeek()((0||1()((0||0(s|07:00|f|17:00)())))(0||2()((0||0(s|07:00|f|17:00)())))(0||3()((0||0(s|07:00|f|17:00)())))(0||4()((0||0(s|07:00|f|17:00)())))(0||5()((0||0(s|07:00|f|17:00)())))(0||6()())(0||7()((0||0(s|07:00|f|17:00)())))))'

In [51]:
new_string = segment[17:-2]
new_string

'((0||1()((0||0(s|07:00|f|17:00)())))(0||2()((0||0(s|07:00|f|17:00)())))(0||3()((0||0(s|07:00|f|17:00)())))(0||4()((0||0(s|07:00|f|17:00)())))(0||5()((0||0(s|07:00|f|17:00)())))(0||6()())(0||7()((0||0(s|07:00|f|17:00)())))'

In [55]:
import pandas as pd
from datetime import timedelta

# Prepare placeholders
days = ['1', '2', '3', '4', '5', '6', '7']
start_hours = ['00:00'] * 7
end_hours = ['00:00'] * 7

# Start from first day
idx = 0
while idx < len(new_string):
    # Find day
    day = new_string[idx:idx + 1]
    if day in days:
        # Get start and end hour
        start_hours[int(day) - 1] = new_string[idx + 21:idx + 26]
        end_hours[int(day) - 1] = new_string[idx + 30:idx + 35]
    idx += 39  # Proceed to next day

# Add working hours
work_hours = [(datetime.strptime(end, '%H:%M') - datetime.strptime(start, '%H:%M')).seconds // 3600 for start, end in
              zip(start_hours, end_hours)]

# Create data frame
dates = pd.date_range("2024-01-01", "2024-12-31")
df = pd.DataFrame(dates, columns=["Day"])
df["Week_Day"] = df["Day"].dt.dayofweek + 1
df["Julian_Date"] = df["Day"].dt.strftime('%Y%j')
df["Starting_Hour"] = df["Week_Day"].apply(lambda x: start_hours[x - 1])
df["Finishing_Hour"] = df["Week_Day"].apply(lambda x: end_hours[x - 1])
df["Working_Hours"] = df["Week_Day"].apply(lambda x: work_hours[x - 1])
df["Working_Day"] = df["Working_Hours"].apply(lambda x: "Yes" if x > 0 else "No")
df["Day"] = df["Day"].dt.strftime('%d-%m-%Y')

print(df)

AttributeError: module 'datetime' has no attribute 'strptime'

In [56]:
import pandas as pd
import datetime

# Prepare placeholders
days = ['1', '2', '3', '4', '5', '6', '7']
start_hours = ['00:00'] * 7
end_hours = ['00:00'] * 7

# Start from first day
idx = 0
while idx < len(new_string):
    # Find day
    day = new_string[idx:idx + 1]
    if day in days:
        # Get start and end hour
        start_hours[int(day) - 1] = new_string[idx + 21:idx + 26]
        end_hours[int(day) - 1] = new_string[idx + 30:idx + 35]
    idx += 39  # Proceed to next day

# Compute working hours
working_hours = [(datetime.datetime.strptime(end, '%H:%M') - datetime.datetime.strptime(start, '%H:%M')).seconds / 3600
                 for start, end in zip(start_hours, end_hours)]

# Build DataFrame for the year 2024
dates = pd.date_range("2024-01-01", "2024-12-31")
df = pd.DataFrame(dates, columns=["Day"])
df["Week_Day"] = df["Day"].dt.dayofweek + 1
df["Julian_Date"] = df["Day"].dt.strftime('%Y%j')
df["Starting_Hour"] = df["Week_Day"].apply(lambda x: start_hours[x - 1])
df["Finishing_Hour"] = df["Week_Day"].apply(lambda x: end_hours[x - 1])
df["Working_Hours"] = df["Week_Day"].apply(lambda x: working_hours[x - 1])
df["Working_Day"] = df["Working_Hours"].apply(lambda x: "Yes" if x > 0 else "No")
df["Day"] = df["Day"].dt.strftime('%d-%m-%Y')

In [57]:
df

,Day,Week_Day,Julian_Date,Starting_Hour,Finishing_Hour,Working_Hours,Working_Day
0,01-01-2024,1,2024001,00:00,00:00,0.0,No
1,02-01-2024,2,2024002,00:00,00:00,0.0,No
2,03-01-2024,3,2024003,00:00,00:00,0.0,No
3,04-01-2024,4,2024004,00:00,00:00,0.0,No
4,05-01-2024,5,2024005,00:00,00:00,0.0,No
5,06-01-2024,6,2024006,00:00,00:00,0.0,No
6,07-01-2024,7,2024007,00:00,00:00,0.0,No
7,08-01-2024,1,2024008,00:00,00:00,0.0,No
8,09-01-2024,2,2024009,00:00,00:00,0.0,No
9,10-01-2024,3,2024010,00:00,00:00,0.0,No


In [60]:
def transform_calendar(clndr_data):
    # Prepare an empty list to collect data
    output = []

    # Iterate over dataframe
    for row in clndr_data:
        # Split the string data
        split_strings = row.split("(0||")

        for split_string in split_strings:
            row_data = {}
            if "DaysOfWeek" in split_string:
                row_data['type'] = "DaysOfWeek"
            elif "Exceptions" in split_string:
                row_data['type'] = "Exceptions"

            if 's|' in split_string and 'f|' in split_string:
                start_time = split_string[split_string.find("s|") + 2:split_string.find("s|") + 6]
                finish_time = split_string[split_string.find("f|") + 2:split_string.find("f|") + 6]

                # Convert string time to datetime.time object
                start_time = datetime.datetime.strptime(start_time, "%H:%M").time()
                finish_time = datetime.datetime.strptime(finish_time, "%H:%M").time()

                row_data['start_time'] = start_time
                row_data['finish_time'] = finish_time

                # Calculate working hours considering the day turn after midnight
                if finish_time > start_time:
                    row_data['working_hours'] = datetime.datetime.combine(datetime.date.today(),
                                                                          finish_time) - datetime.datetime.combine(
                        datetime.date.today(), start_time)
                else:  # the shift ends after midnight
                    row_data['working_hours'] = datetime.datetime.combine(datetime.date.today(),
                                                                          finish_time) + datetime.timedelta(
                        days=1) - datetime.datetime.combine(datetime.date.today(), start_time)

            # Append new row into output list
            output.append(row_data)

    # Convert list of dicts to a DataFrame
    output_df = pd.DataFrame(output)

    return output_df


In [61]:
new_calendar = transform_calendar(clndr_data)

In [62]:
new_calendar


,type,start_time,finish_time,working_hours
0,NaN,NaN,NaN,NaT
1,NaN,NaN,NaN,NaT
2,DaysOfWeek,NaN,NaN,NaT
3,NaN,NaN,NaN,NaT
4,NaN,07:00:00,17:00:00,0 days 10:00:00
5,NaN,NaN,NaN,NaT
6,NaN,07:00:00,17:00:00,0 days 10:00:00
7,NaN,NaN,NaN,NaT
8,NaN,07:00:00,17:00:00,0 days 10:00:00
9,NaN,NaN,NaN,NaT


In [76]:
import pandas as pd
import numpy as np

# Load the calendar data into a DataFrame
calendar_df = pd.read_excel('TST00.xlsx', sheet_name='CALENDAR')

# Find the column index of 'clndr_data'
clndr_data_col = calendar_df.columns.get_loc('clndr_data')

# Initialize an empty DataFrame for Cal_Data
cal_data_columns = ["clndr_id", "Data", "type", "Date", "Hours", "Key"]
cal_data_df = pd.DataFrame(columns=cal_data_columns)

# Counter for the new rows
c = 0

# Iterate over each row in the calendar DataFrame starting from the second row
for i in range(0, len(calendar_df)):
    split_string = calendar_df.iloc[i, clndr_data_col]
    split_target = split_string.split('(0||')
    
    # Create new rows for each split part
    new_rows = pd.DataFrame({
        "clndr_id": [calendar_df.iloc[i, 1]] * len(split_target),
        "Data": split_target
    })
    
    # Append new rows to the Cal_Data DataFrame
    cal_data_df = pd.concat([cal_data_df, new_rows], ignore_index=True)
    c += len(split_target)

# Reset index if needed
cal_data_df.reset_index(drop=True, inplace=True)

# Display the resulting DataFrame
print(cal_data_df)


   clndr_id                           Data type Date Hours  Key
0       993                                 NaN  NaN   NaN  NaN
1       993            CalendarData()(    NaN  NaN   NaN  NaN
2       993            DaysOfWeek()(      NaN  NaN   NaN  NaN
3       993                   1()(        NaN  NaN   NaN  NaN
4       993  0(s|07:00|f|17:00)())))      NaN  NaN   NaN  NaN
5       993                   2()(        NaN  NaN   NaN  NaN
6       993  0(s|07:00|f|17:00)())))      NaN  NaN   NaN  NaN
7       993                   3()(        NaN  NaN   NaN  NaN
8       993  0(s|07:00|f|17:00)())))      NaN  NaN   NaN  NaN
9       993                   4()(        NaN  NaN   NaN  NaN
10      993  0(s|07:00|f|17:00)())))      NaN  NaN   NaN  NaN
11      993                   5()(        NaN  NaN   NaN  NaN
12      993  0(s|07:00|f|17:00)())))      NaN  NaN   NaN  NaN
13      993                   6()())      NaN  NaN   NaN  NaN
14      993                   7()(    

In [73]:
split_string

'(0||CalendarData()(\x7f\x7f  (0||DaysOfWeek()(\x7f\x7f    (0||1()(\x7f\x7f      (0||0(s|07:00|f|17:00)())))\x7f\x7f    (0||2()(\x7f\x7f      (0||0(s|07:00|f|17:00)())))\x7f\x7f    (0||3()(\x7f\x7f      (0||0(s|07:00|f|17:00)())))\x7f\x7f    (0||4()(\x7f\x7f      (0||0(s|07:00|f|17:00)())))\x7f\x7f    (0||5()(\x7f\x7f      (0||0(s|07:00|f|17:00)())))\x7f\x7f    (0||6()())\x7f\x7f    (0||7()(\x7f\x7f      (0||0(s|07:00|f|17:00)())))))\x7f\x7f  (0||VIEW(ShowTotal|Y)())\x7f\x7f  (0||Exceptions()(\x7f\x7f    (0||0(d|44197)(\x7f\x7f      (0||0(s|08:00|f|16:00)())))\x7f\x7f    (0||1(d|44979)())\x7f\x7f    (0||2(d|45038)())\x7f\x7f    (0||3(d|45039)())\x7f\x7f    (0||4(d|45040)())\x7f\x7f    (0||5(d|45041)())\x7f\x7f    (0||6(d|45104)())\x7f\x7f    (0||7(d|45105)())\x7f\x7f    (0||8(d|45106)())\x7f\x7f    (0||9(d|45108)())\x7f\x7f    (0||10(d|45192)())\x7f\x7f    (0||11(d|45344)())\x7f\x7f    (0||12(d|45391)())\x7f\x7f    (0||13(d|45392)())\x7f\x7f    (0||14(d|45393)())\x7f\x7f    (0||15(d|45

In [75]:
split_target

['',
 'CalendarData()(\x7f\x7f  ',
 'DaysOfWeek()(\x7f\x7f    ',
 '1()(\x7f\x7f      ',
 '0(s|07:00|f|17:00)())))\x7f\x7f    ',
 '2()(\x7f\x7f      ',
 '0(s|07:00|f|17:00)())))\x7f\x7f    ',
 '3()(\x7f\x7f      ',
 '0(s|07:00|f|17:00)())))\x7f\x7f    ',
 '4()(\x7f\x7f      ',
 '0(s|07:00|f|17:00)())))\x7f\x7f    ',
 '5()(\x7f\x7f      ',
 '0(s|07:00|f|17:00)())))\x7f\x7f    ',
 '6()())\x7f\x7f    ',
 '7()(\x7f\x7f      ',
 '0(s|07:00|f|17:00)())))))\x7f\x7f  ',
 'VIEW(ShowTotal|Y)())\x7f\x7f  ',
 'Exceptions()(\x7f\x7f    ',
 '0(d|44197)(\x7f\x7f      ',
 '0(s|08:00|f|16:00)())))\x7f\x7f    ',
 '1(d|44979)())\x7f\x7f    ',
 '2(d|45038)())\x7f\x7f    ',
 '3(d|45039)())\x7f\x7f    ',
 '4(d|45040)())\x7f\x7f    ',
 '5(d|45041)())\x7f\x7f    ',
 '6(d|45104)())\x7f\x7f    ',
 '7(d|45105)())\x7f\x7f    ',
 '8(d|45106)())\x7f\x7f    ',
 '9(d|45108)())\x7f\x7f    ',
 '10(d|45192)())\x7f\x7f    ',
 '11(d|45344)())\x7f\x7f    ',
 '12(d|45391)())\x7f\x7f    ',
 '13(d|45392)())\x7f\x7f    ',
 '14(

In [77]:
# Assume cal_data_df is the DataFrame from the previous section

# Function to classify data
def classify_data(data):
    if "DaysOfWeek" in data:
        return "DaysOfWeek"
    elif "Exceptions" in data:
        return "Exceptions"
    else:
        return np.nan

# Apply the classification function to the Data column
cal_data_df['type'] = cal_data_df['Data'].apply(classify_data)

# Display the resulting DataFrame
print(cal_data_df)

   clndr_id                           Data        type Date Hours  Key
0       993                                        NaN  NaN   NaN  NaN
1       993            CalendarData()(           NaN  NaN   NaN  NaN
2       993            DaysOfWeek()(      DaysOfWeek  NaN   NaN  NaN
3       993                   1()(               NaN  NaN   NaN  NaN
4       993  0(s|07:00|f|17:00)())))             NaN  NaN   NaN  NaN
5       993                   2()(               NaN  NaN   NaN  NaN
6       993  0(s|07:00|f|17:00)())))             NaN  NaN   NaN  NaN
7       993                   3()(               NaN  NaN   NaN  NaN
8       993  0(s|07:00|f|17:00)())))             NaN  NaN   NaN  NaN
9       993                   4()(               NaN  NaN   NaN  NaN
10      993  0(s|07:00|f|17:00)())))             NaN  NaN   NaN  NaN
11      993                   5()(               NaN  NaN   NaN  NaN
12      993  0(s|07:00|f|17:00)())))             NaN  NaN   NaN  NaN
13    

In [79]:
from datetime import datetime, timedelta

# Function to extract date and calculate hours
def extract_date_and_hours(row):
    data = row['Data']
    row_type = row['type']
    
    # Initialize date and hours
    date = np.nan
    hours = np.nan

    if row_type == "DaysOfWeek" and data.startswith("()"):
        date = data[1]
    elif row_type == "Exceptions" and "(d|)" in data:
        date = data.split("(d|")[1][:5]

    if "s|" in data and "f|" in data:
        s_index = data.index("s|") + 2
        f_index = data.index("f|") + 2
        start_time_str = data[s_index:s_index + 4]
        end_time_str = data[f_index:f_index + 4]

        # Check if the extracted strings are valid time formats
        if len(start_time_str) == 4 and len(end_time_str) == 4:
            try:
                start_time = datetime.strptime(start_time_str, '%H%M')
                end_time = datetime.strptime(end_time_str, '%H%M')
                if start_time == end_time:
                    hours = 24
                else:
                    hours = (end_time - start_time).seconds / 3600  # Convert to hours
                    if start_time > end_time:
                        hours = 24 - hours
            except ValueError:
                # Handle the case where time conversion fails
                pass

    return pd.Series([date, hours])

# Apply the function to each row
cal_data_df[['Date', 'Hours']] = cal_data_df.apply(extract_date_and_hours, axis=1)

# Fill missing 'type' values with the previous row's 'type'
cal_data_df['type'].fillna(method='ffill', inplace=True)

# Inherit date from previous row if conditions are met
for i in range(1, len(cal_data_df)):
    if pd.isna(cal_data_df.at[i, 'Date']) and cal_data_df.at[i, 'type'] == cal_data_df.at[i-1, 'type'] and \
       cal_data_df.at[i, 'clndr_id'] == cal_data_df.at[i-1, 'clndr_id'] and "VIEW(ShowTotal|Y)" not in cal_data_df.at[i, 'Data']:
        cal_data_df.at[i, 'Date'] = cal_data_df.at[i-1, 'Date']

# Display the resulting DataFrame
print(cal_data_df)


   clndr_id                           Data        type  Date  Hours  Key
0       993                                        NaN   NaN    NaN  NaN
1       993            CalendarData()(           NaN   NaN    NaN  NaN
2       993            DaysOfWeek()(      DaysOfWeek   NaN    NaN  NaN
3       993                   1()(        DaysOfWeek   NaN    NaN  NaN
4       993  0(s|07:00|f|17:00)())))      DaysOfWeek   NaN    NaN  NaN
5       993                   2()(        DaysOfWeek   NaN    NaN  NaN
6       993  0(s|07:00|f|17:00)())))      DaysOfWeek   NaN    NaN  NaN
7       993                   3()(        DaysOfWeek   NaN    NaN  NaN
8       993  0(s|07:00|f|17:00)())))      DaysOfWeek   NaN    NaN  NaN
9       993                   4()(        DaysOfWeek   NaN    NaN  NaN
10      993  0(s|07:00|f|17:00)())))      DaysOfWeek   NaN    NaN  NaN
11      993                   5()(        DaysOfWeek   NaN    NaN  NaN
12      993  0(s|07:00|f|17:00)())))      DaysOfW

C:\Users\abdra\AppData\Local\Temp\ipykernel_15384\1065328383.py:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cal_data_df['type'].fillna(method='ffill', inplace=True)
C:\Users\abdra\AppData\Local\Temp\ipykernel_15384\1065328383.py:44: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cal_data_df['type'].fillna(method='ffill', inplace=True)
